# Load Config

In [1]:
import json
import os

In [ ]:
def load_app_config(config_file):
    config_path = os.path.join(config_file)
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found: {config_path}")
    with open(config_path, "r") as f:
        return json.load(f)

cfg = load_app_config("config_mymlops_dev.json")

# Virtual Machine

In [8]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute.models import (
    HardwareProfile, OSProfile, LinuxConfiguration,
    SshConfiguration, SshPublicKey, NetworkInterfaceReference,
    StorageProfile, ImageReference, OSDisk, DiskCreateOptionTypes
)
from azure.mgmt.network.models import (
    NetworkSecurityGroup, SecurityRule, PublicIPAddress,
    NetworkInterface, Subnet, VirtualNetwork, IPConfiguration
)

In [ ]:
subscription_id = cfg["subscription_id"]
resource_group = cfg["resource_group"]["name"]
location = cfg["resource_group"]["location"]
vm_name = cfg["vm"]["name"]
vm_size = cfg["vm"]["size"]
admin_username = cfg["vm"]["admin_username"]
ssh_key_path = os.path.expanduser(cfg["vm"]["ssh_public_key_path"])
virtual_network = cfg["vm"]["virtual_network"]
sub_network = cfg["vm"]["sub_network"]
network_security_grp = cfg["vm"]["network_security_grp"]
public_ip = cfg["vm"]["public_ip"]
network_interface = cfg["vm"]["network_interface"]

In [ ]:
# ---------------------------------------------------------------------
# Rollback Helper
# ---------------------------------------------------------------------
def rollback_resources(resource_client, network_client, compute_client, rg_name, 
                       created_resources):
    
    for rtype, name in reversed(created_resources):
        try:
            print(f"🗑️  Deleting {rtype}: {name} ...")
            if rtype == "vm":
                compute_client.virtual_machines.begin_delete(rg_name, name).result()
            elif rtype == "nic":
                network_client.network_interfaces.begin_delete(rg_name, name).result()
            elif rtype == "public_ip":
                network_client.public_ip_addresses.begin_delete(rg_name, name).result()
            elif rtype == "nsg":
                network_client.network_security_groups.begin_delete(rg_name, name).result()
            elif rtype == "subnet":
                # Subnet gets deleted with VNet, skip explicit delete
                continue
            elif rtype == "vnet":
                network_client.virtual_networks.begin_delete(rg_name, name).result()
        except Exception as ex:
            print(f"⚠️  Failed to delete {rtype} {name}: {ex}")
    print("✅ Rollback complete.")

In [ ]:
# ---------------------------------------------------------------------
# Function 2: STOP VM (graceful shutdown)
# ---------------------------------------------------------------------
def stop_vm(cfg):
    subscription_id = cfg["azure"]["subscription_id"]
    resource_group = cfg["resource_group"]["name"]
    vm_name = cfg["vm"]["name"]

    credential = DefaultAzureCredential()
    compute_client = ComputeManagementClient(credential, subscription_id)

    print(f"🛑 Stopping VM '{vm_name}'...")
    async_stop = compute_client.virtual_machines.begin_power_off(resource_group, vm_name)
    async_stop.result()
    print(f"✅ VM '{vm_name}' stopped successfully.")


# ---------------------------------------------------------------------
# Function 3: DEALLOCATE VM (stop billing)
# ---------------------------------------------------------------------
def deallocate_vm(cfg):
    
    subscription_id = cfg["azure"]["subscription_id"]
    resource_group = cfg["resource_group"]["name"]
    vm_name = cfg["vm"]["name"]

    credential = DefaultAzureCredential()
    compute_client = ComputeManagementClient(credential, subscription_id)

    print(f"💤 Deallocating VM '{vm_name}' (releases compute)...")
    async_deallocate = compute_client.virtual_machines.begin_deallocate(
        resource_group, vm_name)
    
    async_deallocate.result()
    print(f"✅ VM '{vm_name}' deallocated successfully (no compute cost).")

# Resource Group

In [3]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

In [4]:
rg_config = cfg.get("resource_group", {})
subscription_id = cfg.get("subscription_id", {})
rg_name = rg_config.get("name")
location = rg_config.get("location", "eastus")

In [5]:
def get_resource_client(subscription_id):
    """Authenticate and return the ResourceManagementClient"""
    credential = DefaultAzureCredential()
    return ResourceManagementClient(credential, subscription_id)

In [6]:
resource_client = get_resource_client(subscription_id)

In [7]:
def validate_resource_group(resource_client, rg_name):
    """Check if a resource group exists"""
    print(f"🔍 Validating resource group '{rg_name}'...")
    exists = resource_client.resource_groups.check_existence(rg_name)
    if exists:
        print(f"✅ Resource group '{rg_name}' exists.")
    else:
        print(f"❌ Resource group '{rg_name}' does not exist.")
    return exists

### Deleting Resource group if available

In [ ]:
if validate_resource_group(resource_client, rg_name):
    print(f"🧹 Deleting resource group '{rg_name}'...")
    delete_poller = resource_client.resource_groups.begin_delete(rg_name)
    delete_poller.wait()
    print(f"✅ Resource group '{rg_name}' deleted successfully.")
else:
    print(f"⚠️ Resource group '{rg_name}' does not exist. Nothing to delete.")